### Création du Dataset

#### Partie 3 - Constructions de l'ensemble des fichiers et dossiers pour l'entraînement


In [1]:
%matplotlib inline

import concurrent.futures
import pandas as pd
import os
import sys
from PIL import Image

from IPython.display import display

sys.path.append('..')
import Modules.utils as utils
import Modules.init as init
import Modules.filters as filter

datasets_dir = init.init('Dataset creation')

<br>**Projet M1 - Détection d'anomalies sur imagerie médicale - Dataset creation**

Version              : 2
Notebook id          : Dataset creation
Run time             : Thursday 16 February 2023, 18:33:15
Hostname             : ludovic-ubu (Linux)
Tensorflow log level : Info + Warning + Error  (=0)
Datasets dir         : /media/ludovic/Cache/ENID
Run dir              : ./run
Update keras cache   : False
numpy                : 1.21.5
skimage              : 0.19.2
matplotlib           : 3.5.1
pandas               : 1.3.5


Nous allons créer l'arborecence suivante (avec la proportion de nos fichiers): 
- Train (70 %)
  - Images
  - Masks
- Test (20 %)
  - Images
  - Masks
- Validation (10 %)
  - Images
  - Masks

Création de l'arborescence

In [2]:
root_dir = os.path.join(datasets_dir, 'DATASET')
directories = ['Train', 'Test', 'Validation']
sub_directories = ['Images', 'Masks']

if not os.path.exists(root_dir):
    os.mkdir(root_dir)

for directory in directories:
    dir_path = os.path.join(root_dir, directory)
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    for sub_directory in sub_directories:
        sub_dir_path = os.path.join(dir_path, sub_directory)
        if not os.path.exists(sub_dir_path):
            os.mkdir(sub_dir_path)
        img_dir_path = os.path.join(sub_dir_path, 'img')
        if not os.path.exists(img_dir_path):
            os.mkdir(img_dir_path)

print("Arborescence générée avec succès.")

Arborescence générée avec succès.


Nous allons à présent créer les images qui seront répartis comme définis ci-dessus dans le dataset

In [3]:
# On mélange les lignes de notre fichier et on récupères 3 dataframes
df = pd.read_csv('run/dataset_final.csv', header=0)

train_df, test_df, validate_df = utils.train_validate_test_split(df, train_percent=0.7, validate_percent=0.1, seed=40)

print("Nombre d'images total : ", len(df), "\t\t\tSoit : 100 %")
print("Nombre d'images d'entrainement : ", len(train_df), "\t\tSoit :","{:.2f}".format((len(train_df)*100)/len(df)), " %")
print("Nombre d'images de test : ", len(test_df), "\t\tSoit :","{:.2f}".format((len(test_df)*100)/len(df)), " %")
print("Nombre d'images de validation : ", len(validate_df), "\t\tSoit :","{:.2f}".format((len(validate_df)*100)/len(df)), " %")

Nombre d'images total :  12444 			Soit : 100 %
Nombre d'images d'entrainement :  8710 		Soit : 69.99  %
Nombre d'images de test :  2490 		Soit : 20.01  %
Nombre d'images de validation :  1244 		Soit : 10.00  %


Pour chaque dataframe, nous allons itérer chaque couple de fichier (image + masque) et les répartir dans les sous dossiers correspondant. Nous effecturons le traitement de chaque images ainsi que le redimensionnement si nécessaire.

In [ ]:
path = os.path.join(datasets_dir, 'DATASET')

dataframe_list = [train_df, test_df, validate_df]
folder_list = ['Train', 'Test', 'Validation']
sub_folder_list = ['Images', 'Masks']

def process_case(case):
    folder_path = os.path.join(path, folder_list[case])
    img_path = os.path.join(folder_path, sub_folder_list[0], 'img')
    mask_path = os.path.join(folder_path, sub_folder_list[1], 'img')
    nb_elem = len(dataframe_list[case])
    for i in range(nb_elem):
        utils.progressBar(i, nb_elem, prefix = 'Traitement dataset ' + str(folder_list[case]))
        image_array, mask_array = utils.getImgMsk(dataframe_list[case], i)
        img_stretched = filter.hist_stretching(image_array)
        img_sharped = filter.sharpening(img_stretched, 5, 2.5)
        img_sharped, mask_array = utils.resize_img_mask(384, 640, img_sharped, mask_array)
        mask_array[mask_array == 118] = 1
        
        final_img = Image.fromarray(img_sharped)
        final_mask = Image.fromarray(mask_array)
        
        final_img.save(os.path.join(img_path, str(i) + ".jpeg"))
        final_mask.save(os.path.join(mask_path, str(i) + ".png"))
    print()


In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = [executor.submit(process_case, case) for case in range(3)]
    for f in concurrent.futures.as_completed(results):
        pass